In [37]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

In [38]:
#get and sort train data
t=pd.read_csv('training.csv')
X=t.iloc[:,1:4609]
X=X.values
Y=t.iloc[:,4609].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.5,random_state=42)

In [66]:
#preprocessing (getting best parameters)
RFbeta=RandomForestClassifier()
RFbeata.fit(X_train,Y_train)
paras={'n_estimators':[5,10,25,50,100,250,500],
      'max_depth':[2,4,6,8,16,24,32]}
GSCV=GridSearchCV(RFbeta,paras)
GSCV.fit(X_train,Y_train)
print(GSCV.best_params_)

{'max_depth': 2, 'n_estimators': 10}


In [287]:
#train classifier
RF=RandomForestClassifier()
#**GSCV.best_params_
RF.fit(X_train,Y_train)

RandomForestClassifier()

In [288]:
#analysis of outputs
acc=RF.score(X_test,Y_test)
print('Accuracy =', acc)
p=RF.predict(X_test)
matrix=confusion_matrix(Y_test,p)
print('Matrix',matrix)
PRFS=precision_recall_fscore_support(Y_test,p)
print('         NotMemorable   Memorable')
print('precision',PRFS[0])
print('recall',PRFS[1])
print('f-score',PRFS[2])

Accuracy = 0.6165644171779141
Matrix [[101  75]
 [ 50 100]]
         NotMemorable   Memorable
precision [0.66887417 0.57142857]
recall [0.57386364 0.66666667]
f-score [0.617737   0.61538462]


In [282]:
#get extra training data et=Extra Training, XE0=XExtra0's YE0=YExtra0's, tPlus=training data plus
t=pd.read_csv('training.csv')
et=pd.read_csv('additional_training.csv')
et0=et.loc[et['prediction']==0]
et1=et.loc[et['prediction']==1]
et1= et1.iloc[:117,:]
tPlus=pd.concat([et0,et1,t])

X=tPlus.iloc[:,1:4609]
X=X.values
Y=tPlus.iloc[:,4609].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.5,random_state=42)
print('1s',sum(Y))
print('0s',len(Y)-sum(Y))

1s 326
0s 326


In [283]:
#preprocess extra data
imp=SimpleImputer(missing_values=np.nan,strategy='mean')
imp=imp.fit(X_train)
X_train=imp.transform(X_train)
X_test=imp.transform(X_test)

In [284]:
#get best parameters
betaRF=RandomForestClassifier()
betaRF.fit(X_train,Y_train)
paras={'n_estimators':[943],
      'max_depth':[32]}
GSCV=GridSearchCV(betaRF,paras)
GSCV.fit(X_train,Y_train)
print(GSCV.best_params_)

{'max_depth': 32, 'n_estimators': 943}


In [291]:
RF=RandomForestClassifier(**GSCV.best_params_)
RF.fit(X_train,Y_train)

RandomForestClassifier(max_depth=32, n_estimators=943)

In [292]:
#analysis of outputs
acc=RF.score(X_test,Y_test)
print('Accuracy =', acc)
p=RF.predict(X_test)
matrix=confusion_matrix(Y_test,p)
print('Matrix',matrix)
PRFS=precision_recall_fscore_support(Y_test,p)
print('         NotMemorable   Memorable')
print('precision',PRFS[0])
print('recall',PRFS[1])
print('f-score',PRFS[2])

Accuracy = 0.6625766871165644
Matrix [[113  63]
 [ 47 103]]
         NotMemorable   Memorable
precision [0.70625    0.62048193]
recall [0.64204545 0.68666667]
f-score [0.67261905 0.65189873]


In [267]:
#get testing
test=pd.read_csv('testing.csv')
test=test.values[:,1:]

In [263]:
#run testing through classifier
predictions=pd.DataFrame({"Prediction":RF.predict(test)})
predictions.to_csv('RFprediction15.csv')